In [4]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import torch
import ast

# Load the augmented datasets
train_data = pd.read_csv('train_augmented.csv')
val_data = pd.read_csv('val_augmented.csv')
test_data = pd.read_csv('test_augmented.csv')

# Function to preprocess the data
def preprocess_data(df):
    df['tokens'] = df['tokens'].apply(ast.literal_eval)
    df['text'] = df['tokens'].apply(lambda x: ' '.join(x))
    df['labels'] = df['sentence_label'].astype(int)
    return df[['text', 'labels']]


In [5]:

# Preprocess the data
train_data = preprocess_data(train_data)
val_data = preprocess_data(val_data)
test_data = preprocess_data(test_data)


In [6]:

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Set the format of the datasets
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map: 100%|██████████| 200/200 [00:00<00:00, 7776.66 examples/s]


In [8]:

# Define metrics computation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results_document_classifier",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs_document_classifier',
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()


/Users/burakcivitcioglu/miniconda3/envs/huggingface/lib/python3.8/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  5%|▌         | 10/189 [02:40<46:49, 15.70s/it]

{'loss': 1.5096, 'grad_norm': 4.163740158081055, 'learning_rate': 1.8941798941798943e-05, 'epoch': 0.16}


 11%|█         | 20/189 [05:08<41:06, 14.60s/it]

{'loss': 1.3769, 'grad_norm': 9.211262702941895, 'learning_rate': 1.7883597883597884e-05, 'epoch': 0.32}


 16%|█▌        | 30/189 [07:29<37:19, 14.09s/it]

...

                                                 
100%|██████████| 189/189 [47:32<00:00, 11.71s/it]

{'eval_loss': 0.5309481024742126, 'eval_accuracy': 0.845, 'eval_f1': 0.8370239563567364, 'eval_runtime': 49.9777, 'eval_samples_per_second': 4.002, 'eval_steps_per_second': 0.26, 'epoch': 3.0}


100%|██████████| 189/189 [47:38<00:00, 15.12s/it]


{'train_runtime': 2858.4231, 'train_samples_per_second': 1.05, 'train_steps_per_second': 0.066, 'train_loss': 0.8244055112202963, 'epoch': 3.0}


TrainOutput(global_step=189, training_loss=0.8244055112202963, metrics={'train_runtime': 2858.4231, 'train_samples_per_second': 1.05, 'train_steps_per_second': 0.066, 'total_flos': 197338606848000.0, 'train_loss': 0.8244055112202963, 'epoch': 3.0})

In [9]:

# Evaluate on the test set
test_results = trainer.evaluate(tokenized_test)

print("Test results:", test_results)

# Save the model
trainer.save_model("./document_classifier_model")
print("Model saved to ./document_classifier_model")

100%|██████████| 13/13 [00:47<00:00,  3.69s/it]


Test results: {'eval_loss': 0.6955267190933228, 'eval_accuracy': 0.755, 'eval_f1': 0.7386042154566745, 'eval_runtime': 52.0812, 'eval_samples_per_second': 3.84, 'eval_steps_per_second': 0.25, 'epoch': 3.0}
Model saved to ./document_classifier_model
